In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import json
import pandas as pd
from nltk import word_tokenize
import gensim 
from gensim.models import Word2Vec
import re
import csv
import sys
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
import spacy
import en_core_web_sm
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr4/ugrad/nshalash/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
#Merging all the txt created by "Articles by Race" code into one txt.
import glob

read_files = glob.glob("*.txt")

with open("result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())
            outfile.write("\n")

In [2]:
data_df = open("result.txt", "r",encoding="utf-8").readlines()
for i in range(len(data_df)):
    data_df[i] = str(data_df[i]).replace('[','').replace(']','')
    data_df[i] = str(data_df[i]).replace('\'','')
    

# Topic Modelling

In [3]:
#nlp = spacy.load('en_core_web_lg')
nlp = en_core_web_sm.load()
matcher = PhraseMatcher(nlp.vocab)

In [5]:
data = list(dict.fromkeys(data_df))

In [6]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 'year', 'movie', 'game', 'bruin', 'play', 'time', 'people', 'team', 'show','feel','day','still','well','way','re', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come','said','tell','ask'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr4/ugrad/nshalash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  
    
data_words = list(sent_to_words(data))


In [9]:
nlp.max_length = 10000000000000

In [10]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
# or do
# !conda install -c conda-forge spacy-model-en_core_web_md 
# and use nlp=spacy.load('en_core_web_sm') instead in below function.
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)

In [11]:
print(len(data_words))

1


In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

print(lda_model.print_topics())

[(0, '0.000*"black" + 0.000*"last" + 0.000*"season" + 0.000*"first" + 0.000*"young" + 0.000*"player" + 0.000*"woman" + 0.000*"new" + 0.000*"look" + 0.000*"man"'), (1, '0.000*"black" + 0.000*"work" + 0.000*"look" + 0.000*"new" + 0.000*"last" + 0.000*"first" + 0.000*"man" + 0.000*"include" + 0.000*"woman" + 0.000*"player"'), (2, '0.006*"black" + 0.004*"last" + 0.004*"first" + 0.003*"work" + 0.003*"look" + 0.003*"new" + 0.003*"season" + 0.003*"woman" + 0.003*"man" + 0.003*"player"'), (3, '0.000*"black" + 0.000*"last" + 0.000*"work" + 0.000*"season" + 0.000*"first" + 0.000*"look" + 0.000*"end" + 0.000*"turn" + 0.000*"give" + 0.000*"player"'), (4, '0.000*"black" + 0.000*"first" + 0.000*"woman" + 0.000*"work" + 0.000*"last" + 0.000*"student" + 0.000*"much" + 0.000*"new" + 0.000*"man" + 0.000*"big"'), (5, '0.001*"black" + 0.000*"first" + 0.000*"last" + 0.000*"woman" + 0.000*"look" + 0.000*"back" + 0.000*"old" + 0.000*"call" + 0.000*"work" + 0.000*"man"')]


In [ ]:
#Only works when number of topics are 4 in above cell(num_topics)

from collections import Counter
import matplotlib.colors as mcolors
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 7500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=10)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count for White Community', fontsize=22, y=1.05)    
plt.show()

In [13]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis
#pyLDAvis.save_html(vis, 'race_data.html')

/usr4/ugrad/nshalash/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
